## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Yellowknife,62.4560,-114.3525,30.42,88,40,7.00,CA,2021-10-30 18:36:23
1,1,Port Moresby,-9.4431,147.1797,74.62,94,20,8.05,PG,2021-10-30 18:38:31
2,2,Naze,28.3667,129.4833,67.37,78,0,6.62,JP,2021-10-30 18:35:03
3,3,Ushuaia,-54.8000,-68.3000,49.66,53,0,18.41,AR,2021-10-30 18:38:32
4,4,Atuona,-9.8000,-139.0333,77.76,67,6,19.10,PF,2021-10-30 18:38:32


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
4,4,Atuona,-9.8000,-139.0333,77.76,67,6,19.10,PF,2021-10-30 18:38:32
6,6,Veinticinco De Mayo,-35.4323,-60.1727,92.98,29,3,1.99,AR,2021-10-30 18:38:32
9,9,Bell Ville,-32.6259,-62.6887,96.80,19,3,8.75,AR,2021-10-30 18:34:38
11,11,Hilo,19.7297,-155.0900,77.25,66,1,6.91,US,2021-10-30 18:36:19
28,28,Kununurra,-15.7667,128.7333,83.28,64,84,1.21,AU,2021-10-30 18:38:38


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

Unnamed: 0    241
City          241
Lat           241
Lng           241
Max Temp      241
Humidity      241
Cloudiness    241
Wind Speed    241
Country       240
Date          241
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Date", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Date,Lat,Lng,Hotel Name
4,Atuona,PF,77.76,2021-10-30 18:38:32,-9.8000,-139.0333,
6,Veinticinco De Mayo,AR,92.98,2021-10-30 18:38:32,-35.4323,-60.1727,
9,Bell Ville,AR,96.80,2021-10-30 18:34:38,-32.6259,-62.6887,
11,Hilo,US,77.25,2021-10-30 18:36:19,19.7297,-155.0900,
28,Kununurra,AU,83.28,2021-10-30 18:38:38,-15.7667,128.7333,
29,Tucurui,BR,92.12,2021-10-30 18:38:38,-3.7661,-49.6725,
30,Butaritari,KI,82.27,2021-10-30 18:38:38,3.0707,172.7902,
31,Barra,BR,90.64,2021-10-30 18:38:39,-11.0894,-43.1417,
33,Cabo San Lucas,MX,87.51,2021-10-30 18:38:39,22.8909,-109.9124,
48,Port Blair,IN,80.55,2021-10-30 18:37:47,11.6667,92.7500,


In [18]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat=row['Lat']
    lng= row['Lng']
    
    params["location"] =f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.    
    try:
        hotel_df.loc[index, "Hotel Name"] = results[0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        hotel_df.loc[i, "Hotel Name"]=np.nan
        
        

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [19]:
hotel_df.head(10)

,City,Country,Max Temp,Date,Lat,Lng,Hotel Name
4,Atuona,PF,77.76,2021-10-30 18:38:32,-9.8000,-139.0333,Villa Enata
6,Veinticinco De Mayo,AR,92.98,2021-10-30 18:38:32,-35.4323,-60.1727,Estancia La Rosada
9,Bell Ville,AR,96.80,2021-10-30 18:34:38,-32.6259,-62.6887,Hotel Central
11,Hilo,US,77.25,2021-10-30 18:36:19,19.7297,-155.0900,Hilo Hawaiian Hotel
28,Kununurra,AU,83.28,2021-10-30 18:38:38,-15.7667,128.7333,Hotel Kununurra
29,Tucurui,BR,92.12,2021-10-30 18:38:38,-3.7661,-49.6725,BBB Rooms Centro Tucurui PA
30,Butaritari,KI,82.27,2021-10-30 18:38:38,3.0707,172.7902,Isles Sunset Lodge
31,Barra,BR,90.64,2021-10-30 18:38:39,-11.0894,-43.1417,Motel Albatroz
33,Cabo San Lucas,MX,87.51,2021-10-30 18:38:39,22.8909,-109.9124,Hotel Tesoro Los Cabos
48,Port Blair,IN,80.55,2021-10-30 18:37:47,11.6667,92.7500,"Welcomhotel By ITC Hotels, Bay Island, Port Blair"


In [20]:
# 7. Drop the rows where there is no Hotel Name.
try:
    clean_hotel_df = hotel_df.dropna()

except:
    print('No empty rows to drop')
    clean_hotel_df = hotel_df

In [21]:
clean_hotel_df.head()

,City,Country,Max Temp,Date,Lat,Lng,Hotel Name
4,Atuona,PF,77.76,2021-10-30 18:38:32,-9.8000,-139.0333,Villa Enata
6,Veinticinco De Mayo,AR,92.98,2021-10-30 18:38:32,-35.4323,-60.1727,Estancia La Rosada
9,Bell Ville,AR,96.80,2021-10-30 18:34:38,-32.6259,-62.6887,Hotel Central
11,Hilo,US,77.25,2021-10-30 18:36:19,19.7297,-155.0900,Hilo Hawaiian Hotel
28,Kununurra,AU,83.28,2021-10-30 18:38:38,-15.7667,128.7333,Hotel Kununurra


In [23]:
# 8a. Create the output File (CSV)
output_data_file= 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [25]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt><b>Hotel Name</b></dt><dd>{Hotel Name}</dd>
<dt><b>City</b></dt><dd>{City}</dd>
<dt><b>Country</b></dt><dd>{Country}</dd>
<dt><b>Weather</b></dt><dd>{Date}</dd>
<dt><b>Max Temp</b></dt><dd>{Max Temp}</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [26]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)

fig.add_layer(heat_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))